In [44]:
import numpy as np

In [45]:
R=13
A = (np.random.random((R, R)) * 10).round(2)
b = np.random.randint(1, 10, size=(R, 1))
B=b.copy()

In [46]:
np.set_printoptions(precision=4, suppress=True, linewidth=np.inf)

In [47]:
import time

### Algorytm eliminacji Gaussa bez pivotingu generujący jedynki na przekątnej

In [48]:
def gauss_elimination(A, b):
    
    A = A.copy().astype(float)
    b = b.copy().astype(float)
    n = len(b)

    # Eliminacja w przód
    for i in range(n):
        # Tworzymy 1 na przekątnej przez dzielenie całego wiersza
        diagonal = A[i, i]
        A[i] = A[i] / diagonal
        b[i] = b[i] / diagonal

        # Eliminacja elementów poniżej
        for j in range(i+1, n):
            multiplier = A[j, i]
            A[j] = A[j] - multiplier * A[i]
            b[j] = b[j] - multiplier * b[i]

    # Podstawianie wsteczne
    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        x[i] = (b[i] - np.dot(A[i, i+1:], x[i+1:])).item()

    return x

In [49]:
start = time.time()    
x = gauss_elimination(A, b)
display("Rozwiązanie:", x)
end = time.time()   

'Rozwiązanie:'

array([-1.6949,  1.8708,  0.7768,  1.0932, -0.3524,  2.0929,  1.1428, -0.332 ,  0.3586, -1.5577, -1.0116, -1.2818,  0.2741])

In [50]:
b_uzyskane = A @ x
display(b_uzyskane)
display("Rozwiązanie:", x)
print(f"Czas wykonania: {end - start:.6f} sekund")


array([4., 9., 3., 2., 9., 7., 2., 7., 8., 8., 2., 9., 7.])

'Rozwiązanie:'

array([-1.6949,  1.8708,  0.7768,  1.0932, -0.3524,  2.0929,  1.1428, -0.332 ,  0.3586, -1.5577, -1.0116, -1.2818,  0.2741])

Czas wykonania: 0.004109 sekund


In [51]:
display(b.flatten())

array([4, 9, 3, 2, 9, 7, 2, 7, 8, 8, 2, 9, 7])

In [52]:
display(A)

array([[0.1 , 8.29, 8.73, 1.88, 4.79, 3.82, 0.43, 4.5 , 9.85, 6.57, 6.49, 9.75, 1.04],
       [4.8 , 6.9 , 0.34, 6.41, 4.89, 1.95, 2.34, 0.72, 2.07, 0.69, 1.8 , 6.56, 9.94],
       [8.93, 7.73, 6.27, 5.16, 0.62, 1.52, 8.71, 6.65, 8.35, 6.23, 3.85, 6.11, 3.25],
       [6.56, 8.3 , 3.79, 9.55, 8.17, 8.42, 2.18, 8.33, 6.2 , 9.64, 5.57, 9.63, 1.86],
       [1.48, 3.52, 0.09, 9.5 , 1.91, 3.55, 7.15, 0.25, 7.65, 5.43, 3.91, 8.96, 2.84],
       [8.32, 8.33, 5.51, 1.15, 8.81, 4.44, 6.26, 6.02, 6.73, 0.84, 7.35, 3.96, 0.16],
       [9.21, 0.75, 8.72, 9.61, 5.41, 6.3 , 0.07, 8.21, 7.81, 0.12, 6.19, 6.34, 7.55],
       [3.13, 7.91, 1.35, 7.44, 4.09, 1.53, 5.55, 5.77, 9.05, 6.25, 2.85, 7.01, 1.81],
       [5.1 , 2.29, 7.79, 3.48, 3.13, 9.77, 4.54, 8.75, 4.14, 5.11, 8.14, 5.19, 8.17],
       [9.28, 3.74, 9.97, 4.77, 7.5 , 8.98, 3.79, 0.17, 3.88, 7.21, 6.96, 1.61, 8.36],
       [2.66, 6.95, 8.  , 0.64, 2.71, 2.13, 8.13, 4.69, 1.38, 8.05, 2.98, 8.23, 3.52],
       [9.67, 2.75, 6.91, 2.84, 5.17, 8.7 ,

### Algorytm eliminacji Gaussa z pivotingiem

In [53]:

def gauss_elimination_with_partial_pivoting(A, b):
    """
    Eliminacja Gaussa z częściowym pivotingiem.
    
    A: ndarray kształtu (n, n)
    b: ndarray kształtu (n,)
    
    Zwraca wektor x (ndarray) o długości n, będący rozwiązaniem A x = b.
    """
    A = A.copy().astype(float)
    b = b.copy().astype(float).reshape(-1)
    n = len(b)

    # Eliminacja w przód
    for i in range(n):
        # 1) Znajdź wiersz z największym (modułowo) elementem w kolumnie i (od wiersza i do n-1)
        max_row = i + np.argmax(np.abs(A[i:, i]))

        # 2) Zamiana wierszy i <-> max_row (jeśli różne)
        if max_row != i:
            A[[i, max_row], :] = A[[max_row, i], :]
            b[i], b[max_row] = b[max_row], b[i]

        # 3) Wyzerowanie elementów w kolumnie i, w wierszach poniżej i
        pivot = A[i, i]
        for j in range(i+1, n):
            factor = A[j, i] / pivot
            A[j, i:] -= factor * A[i, i:]
            b[j] -= factor * float(b[i])

    # Podstawianie wsteczne
    x = np.zeros(n, dtype=float)
    for i in range(n-1, -1, -1):
        x[i] = ((b[i] - np.dot(A[i, i+1:], x[i+1:])) / A[i, i]).item()

    return x

In [54]:
start = time.time()    
x = gauss_elimination_with_partial_pivoting(A, b)
end = time.time()   
display("Rozwiązanie:", x)
print(f"Czas wykonania: {end - start:.6f} sekund")

'Rozwiązanie:'

array([-1.6949,  1.8708,  0.7768,  1.0932, -0.3524,  2.0929,  1.1428, -0.332 ,  0.3586, -1.5577, -1.0116, -1.2818,  0.2741])

Czas wykonania: 0.001004 sekund


In [55]:
display(A)

array([[0.1 , 8.29, 8.73, 1.88, 4.79, 3.82, 0.43, 4.5 , 9.85, 6.57, 6.49, 9.75, 1.04],
       [4.8 , 6.9 , 0.34, 6.41, 4.89, 1.95, 2.34, 0.72, 2.07, 0.69, 1.8 , 6.56, 9.94],
       [8.93, 7.73, 6.27, 5.16, 0.62, 1.52, 8.71, 6.65, 8.35, 6.23, 3.85, 6.11, 3.25],
       [6.56, 8.3 , 3.79, 9.55, 8.17, 8.42, 2.18, 8.33, 6.2 , 9.64, 5.57, 9.63, 1.86],
       [1.48, 3.52, 0.09, 9.5 , 1.91, 3.55, 7.15, 0.25, 7.65, 5.43, 3.91, 8.96, 2.84],
       [8.32, 8.33, 5.51, 1.15, 8.81, 4.44, 6.26, 6.02, 6.73, 0.84, 7.35, 3.96, 0.16],
       [9.21, 0.75, 8.72, 9.61, 5.41, 6.3 , 0.07, 8.21, 7.81, 0.12, 6.19, 6.34, 7.55],
       [3.13, 7.91, 1.35, 7.44, 4.09, 1.53, 5.55, 5.77, 9.05, 6.25, 2.85, 7.01, 1.81],
       [5.1 , 2.29, 7.79, 3.48, 3.13, 9.77, 4.54, 8.75, 4.14, 5.11, 8.14, 5.19, 8.17],
       [9.28, 3.74, 9.97, 4.77, 7.5 , 8.98, 3.79, 0.17, 3.88, 7.21, 6.96, 1.61, 8.36],
       [2.66, 6.95, 8.  , 0.64, 2.71, 2.13, 8.13, 4.69, 1.38, 8.05, 2.98, 8.23, 3.52],
       [9.67, 2.75, 6.91, 2.84, 5.17, 8.7 ,

In [56]:
b_uzyskane = A @ x
display(b_uzyskane)


array([4., 9., 3., 2., 9., 7., 2., 7., 8., 8., 2., 9., 7.])

In [57]:
display(b)

array([[4],
       [9],
       [3],
       [2],
       [9],
       [7],
       [2],
       [7],
       [8],
       [8],
       [2],
       [9],
       [7]])

### Algorytm LU faktoryzacji bez pivotingu

In [58]:
np.set_printoptions()

In [59]:

from IPython.display import display

#A = (np.random.random(9).reshape((3,3))*10).round()
#B = (np.ones(3).reshape((3,1))*10).round()

"""A = np.array([
    [1, 5, 2],
    [1, 2, -3],
    [2, 4, 1]
], dtype=float)

B = np.array([
    [1],
    [-5],
    [4]
], dtype=float)
"""
start = time.time()   
O = np.identity(R)

# Początkowe dane
#display("Macierz jednostkowa (startowa L):", O)
#display("Macierz A:", A)
#display("Wektor B:", B)

A_work = A.copy().astype(float)
B_work = B.copy().astype(float)
x1 = np.linalg.solve(A, B)
n = len(A)

# Eliminacja Gaussa (bez pivotingu)
for i in range(n-1):
    for j in range(i+1,n):
        div = (A_work[j,i]/A_work[i,i])
        # display(f"Dzielnik dla A[{j},{i}]/A[{i},{i}] = {div}")
        # display("Mnożnik (L-row):", O[i]*div)
        O[j,i] = div
        A_work[j] -= A_work[i]*div
        # display("Aktualna macierz L:", O)

# Zaokrąglanie zer bliskich 0
A_work[abs(A_work) < 0.00001] = 0.0

# L i U gotowe
display("Macierz L (dolna trójkątna):", O)
display("Macierz U (górna trójkątna):", A_work)
#display("Wektor B:", B)
display("Rozwiązanie z numpy.linalg.solve (kontrola):", x1)

L = O
U = A_work

# mamy L i U - koniec, finito
C = np.zeros((R,1))
C[0] = B_work[0]
for i in range(1,n):
    known_c_sum = 0
    for j in range(0,i):
        known_c_sum += L[i,j] * C[j]
    C[i] = B_work[i]-known_c_sum

# Podstawianie wsteczne
x = np.zeros((R,1))
x[n-1,0] = C[n-1,0]/U[n-1,n-1]

for i in range(n-2,-1,-1):
    known_xu_sum = 0
    for j in range(i+1,n):
        known_xu_sum += U[i,j] * x[j,0]
    x[i,0] = (C[i,0]-known_xu_sum)/U[i,i]
end = time.time()  
# Wyniki końcowe
display("Rozwiązanie x:", x)
display("Sprawdzenie: L @ U @ x =", L @ U @ x)
display("Oryginalny wektor B:", B)
print(f"Czas wykonania: {end - start:.6f} sekund")

'Macierz L (dolna trójkątna):'

array([[ 1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [48.    ,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [89.3   ,  1.8735,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [65.6   ,  1.3696,  0.4084,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [14.8   ,  0.3048, -0.1356,  1.9286,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [83.2   ,  1.7426,  0.7931, -1.3997, -1.3465,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [92.1   ,  1.9507,  1.9306,  3.2668,  0.0481,  6.9544,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0.    ],
       [31.3   ,  0.6434, -0.2272,  0.3705,  0.41  , -0.2174, -0.0399,  1.    ,  0.    ,  0.    ,  0.    ,  0.    ,  0

'Macierz U (górna trójkątna):'

array([[   0.1   ,    8.29  ,    8.73  ,    1.88  ,    4.79  ,    3.82  ,    0.43  ,    4.5   ,    9.85  ,    6.57  ,    6.49  ,    9.75  ,    1.04  ],
       [   0.    , -391.02  , -418.7   ,  -83.83  , -225.03  , -181.41  ,  -18.3   , -215.28  , -470.73  , -314.67  , -309.72  , -461.44  ,  -39.98  ],
       [   0.    ,    0.    ,   11.1058,   -5.6704,   -5.5385,    0.2615,    4.5956,    8.1221,   10.6469,    9.056 ,    4.5463,   -0.0677,  -14.7204],
       [   0.    ,    0.    ,    0.    ,    3.3476,    4.3991,    6.1725,   -2.8418,    4.6513,    0.3835,    5.9081,    2.1485,    2.0258,   -5.5977],
       [   0.    ,    0.    ,    0.    ,    0.    ,   -9.6342,   -9.5661,   12.4673,   -8.6078,    6.0395,   -6.0699,   -1.2751,    1.3778,    8.4326],
       [   0.    ,    0.    ,    0.    ,    0.    ,    0.    ,   -1.7048,   11.539 ,   -4.7512,    7.7373,   -4.5206,    4.79  ,    1.6175,   -1.5043],
       [   0.    ,    0.    ,    0.    ,    0.    ,    0.    ,    0.    ,  -84.2708,   1

'Rozwiązanie z numpy.linalg.solve (kontrola):'

...

'Sprawdzenie: L @ U @ x ='

array([[4.],
       [9.],
       [3.],
       [2.],
       [9.],
       [7.],
       [2.],
       [7.],
       [8.],
       [8.],
       [2.],
       [9.],
       [7.]])

'Oryginalny wektor B:'

array([[4],
       [9],
       [3],
       [2],
       [9],
       [7],
       [2],
       [7],
       [8],
       [8],
       [2],
       [9],
       [7]])

Czas wykonania: 0.011038 sekund


### Algorytm LU faktoryzacji z pivotingiem

In [60]:
import numpy as np
from IPython.display import display
start = time.time()
def lu_no_pivot(A):
    n = A.shape[0]
    L = np.identity(n)
    U = A.copy().astype(float)

    for i in range(n-1):
        for j in range(i+1,n):
            div = (U[j,i]/U[i,i])
            L[j,i] = div
            U[j] -= U[i]*div
    return L, U


def lu_pivot(A):
    n = A.shape[0]
    A = A.copy().astype(float)

    arr = [np.identity(n)]
    U = A.copy()

    for i in range(n-1):
        P2 = np.identity(n)
        m = np.argmax(np.abs(U[i:,i]), axis=0) + i  # pivot wybierany na podstawie |elementów|
        U[[i, m]] = U[[m, i]]  # zamiana wierszy
        P2[[i, m]] = P2[[m, i]]
        arr.append(P2)

        P3 = np.identity(n)
        for j in range(i+1,n):
            div = (U[j,i]/U[i,i])
            P3[j,i] = div
            U[j] -= U[i]*div
        arr.append(P3)

    P = np.identity(n)
    for i in range(len(arr)-2, -1, -2):
        P = P @ arr[i]

    L = np.identity(n)
    for i in range(1, len(arr), 1):
        L = L @ arr[i]
    L = P @ L

    return P, L, U


"""A_screen = np.array([
    [0, 4, 1],
    [1, 1, 3],
    [2, -2, 1]
], dtype=float)
"""
P, L, U = lu_pivot(A)
end = time.time() 
display("Macierz A (oryginalna):", A)
display("Macierz permutacji P:", P)
display("Macierz dolna L:", L)
display("Macierz górna U:", U)

display("Weryfikacja: P @ A =")
display(P @ A)
display("Weryfikacja: L @ U =")
display(L @ U)
print(f"Czas wykonania: {end - start:.6f} sekund")

'Macierz A (oryginalna):'

array([[0.1 , 8.29, 8.73, 1.88, 4.79, 3.82, 0.43, 4.5 , 9.85, 6.57, 6.49, 9.75, 1.04],
       [4.8 , 6.9 , 0.34, 6.41, 4.89, 1.95, 2.34, 0.72, 2.07, 0.69, 1.8 , 6.56, 9.94],
       [8.93, 7.73, 6.27, 5.16, 0.62, 1.52, 8.71, 6.65, 8.35, 6.23, 3.85, 6.11, 3.25],
       [6.56, 8.3 , 3.79, 9.55, 8.17, 8.42, 2.18, 8.33, 6.2 , 9.64, 5.57, 9.63, 1.86],
       [1.48, 3.52, 0.09, 9.5 , 1.91, 3.55, 7.15, 0.25, 7.65, 5.43, 3.91, 8.96, 2.84],
       [8.32, 8.33, 5.51, 1.15, 8.81, 4.44, 6.26, 6.02, 6.73, 0.84, 7.35, 3.96, 0.16],
       [9.21, 0.75, 8.72, 9.61, 5.41, 6.3 , 0.07, 8.21, 7.81, 0.12, 6.19, 6.34, 7.55],
       [3.13, 7.91, 1.35, 7.44, 4.09, 1.53, 5.55, 5.77, 9.05, 6.25, 2.85, 7.01, 1.81],
       [5.1 , 2.29, 7.79, 3.48, 3.13, 9.77, 4.54, 8.75, 4.14, 5.11, 8.14, 5.19, 8.17],
       [9.28, 3.74, 9.97, 4.77, 7.5 , 8.98, 3.79, 0.17, 3.88, 7.21, 6.96, 1.61, 8.36],
       [2.66, 6.95, 8.  , 0.64, 2.71, 2.13, 8.13, 4.69, 1.38, 8.05, 2.98, 8.23, 3.52],
       [9.67, 2.75, 6.91, 2.84, 5.17, 8.7 ,

'Macierz permutacji P:'

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.]])

'Macierz dolna L:'

...

'Weryfikacja: P @ A ='

array([[9.67, 2.75, 6.91, 2.84, 5.17, 8.7 , 2.14, 7.28, 6.29, 4.46, 1.45, 0.01, 5.58],
       [0.1 , 8.29, 8.73, 1.88, 4.79, 3.82, 0.43, 4.5 , 9.85, 6.57, 6.49, 9.75, 1.04],
       [8.49, 9.44, 3.76, 2.6 , 7.86, 2.02, 9.41, 1.87, 6.9 , 6.89, 2.35, 3.79, 6.82],
       [1.48, 3.52, 0.09, 9.5 , 1.91, 3.55, 7.15, 0.25, 7.65, 5.43, 3.91, 8.96, 2.84],
       [8.93, 7.73, 6.27, 5.16, 0.62, 1.52, 8.71, 6.65, 8.35, 6.23, 3.85, 6.11, 3.25],
       [9.21, 0.75, 8.72, 9.61, 5.41, 6.3 , 0.07, 8.21, 7.81, 0.12, 6.19, 6.34, 7.55],
       [6.56, 8.3 , 3.79, 9.55, 8.17, 8.42, 2.18, 8.33, 6.2 , 9.64, 5.57, 9.63, 1.86],
       [9.28, 3.74, 9.97, 4.77, 7.5 , 8.98, 3.79, 0.17, 3.88, 7.21, 6.96, 1.61, 8.36],
       [2.66, 6.95, 8.  , 0.64, 2.71, 2.13, 8.13, 4.69, 1.38, 8.05, 2.98, 8.23, 3.52],
       [4.8 , 6.9 , 0.34, 6.41, 4.89, 1.95, 2.34, 0.72, 2.07, 0.69, 1.8 , 6.56, 9.94],
       [8.32, 8.33, 5.51, 1.15, 8.81, 4.44, 6.26, 6.02, 6.73, 0.84, 7.35, 3.96, 0.16],
       [3.13, 7.91, 1.35, 7.44, 4.09, 1.53,

'Weryfikacja: L @ U ='

array([[9.67, 2.75, 6.91, 2.84, 5.17, 8.7 , 2.14, 7.28, 6.29, 4.46, 1.45, 0.01, 5.58],
       [0.1 , 8.29, 8.73, 1.88, 4.79, 3.82, 0.43, 4.5 , 9.85, 6.57, 6.49, 9.75, 1.04],
       [8.49, 9.44, 3.76, 2.6 , 7.86, 2.02, 9.41, 1.87, 6.9 , 6.89, 2.35, 3.79, 6.82],
       [1.48, 3.52, 0.09, 9.5 , 1.91, 3.55, 7.15, 0.25, 7.65, 5.43, 3.91, 8.96, 2.84],
       [8.93, 7.73, 6.27, 5.16, 0.62, 1.52, 8.71, 6.65, 8.35, 6.23, 3.85, 6.11, 3.25],
       [9.21, 0.75, 8.72, 9.61, 5.41, 6.3 , 0.07, 8.21, 7.81, 0.12, 6.19, 6.34, 7.55],
       [6.56, 8.3 , 3.79, 9.55, 8.17, 8.42, 2.18, 8.33, 6.2 , 9.64, 5.57, 9.63, 1.86],
       [9.28, 3.74, 9.97, 4.77, 7.5 , 8.98, 3.79, 0.17, 3.88, 7.21, 6.96, 1.61, 8.36],
       [2.66, 6.95, 8.  , 0.64, 2.71, 2.13, 8.13, 4.69, 1.38, 8.05, 2.98, 8.23, 3.52],
       [4.8 , 6.9 , 0.34, 6.41, 4.89, 1.95, 2.34, 0.72, 2.07, 0.69, 1.8 , 6.56, 9.94],
       [8.32, 8.33, 5.51, 1.15, 8.81, 4.44, 6.26, 6.02, 6.73, 0.84, 7.35, 3.96, 0.16],
       [3.13, 7.91, 1.35, 7.44, 4.09, 1.53,

Czas wykonania: 0.001025 sekund


In [61]:
b_list = [np.random.randint(1, 10, size=(R, 1)) for _ in range(4)]

In [62]:

def test_gauss_for_multiple_b(A, b_list):
    
    total_time = 0.0

    for i, b in enumerate(b_list):
        start = time.time()
        x = gauss_elimination(A, b)
        end = time.time()
        elapsed = end - start
        total_time += elapsed

        print(f"Rozwiązanie dla b[{i+1}]:")
        print(x)
        print(f"Czas: {elapsed:.6f} sekundy\n")

    print(f" Suma czasów: {total_time:.6f} sekundy")
    return total_time


In [63]:
total_time = test_gauss_for_multiple_b(A, b_list)
print(total_time)

Rozwiązanie dla b[1]:
[-0.1709 -0.2575  0.2755  0.5809  0.4728 -1.1799  0.0179  0.7365  0.1063  0.4832  0.3182 -0.3988  0.1697]
Czas: 0.000000 sekundy

Rozwiązanie dla b[2]:
[-0.7267  1.4345 -0.0639 -0.2173 -1.1062  1.4376  0.1101 -0.0906  0.7209 -0.7055 -0.5219 -0.1577  0.7431]
Czas: 0.001999 sekundy

Rozwiązanie dla b[3]:
[ 0.8826 -0.3852 -0.7122 -0.7112 -0.2823 -0.5304 -0.5948  0.2248  0.5532  0.5228  0.3034  0.5845  0.7023]
Czas: 0.000000 sekundy

Rozwiązanie dla b[4]:
[-0.1146  0.3546  0.0785 -0.0962  0.4364  0.8578 -0.0581 -0.4461  0.3108 -0.1039 -0.6393  0.1873  0.1705]
Czas: 0.001426 sekundy

 Suma czasów: 0.003426 sekundy
0.00342559814453125


In [66]:
def lu_solve_multiple_b(A: np.ndarray, b_list: list[np.ndarray]) -> float:
    start = time.time()

    R = A.shape[0]
    A_work = A.copy().astype(float)
    O = np.identity(R)

    # 1. ROZKŁAD LU bez pivotingu (tylko raz)
    for i in range(R - 1):
        for j in range(i + 1, R):
            div = A_work[j, i] / A_work[i, i]
            O[j, i] = div
            A_work[j] -= A_work[i] * div

    L = O
    U = A_work

    # 2. Dla każdego wektora b rozwiązujemy LUx = b
    for idx, B in enumerate(b_list):
        B_work = B.copy().astype(float)

        # a) Rozwiązanie Lc = b (przekształcenie w dół)
        C = np.zeros((R, 1))
        C[0] = B_work[0]
        for i in range(1, R):
            known_c_sum = np.dot(L[i, :i], C[:i, 0])
            C[i] = B_work[i] - known_c_sum

        # b) Rozwiązanie Ux = c (podstawianie wsteczne)
        x = np.zeros((R, 1))
        x[-1, 0] = C[-1, 0] / U[-1, -1]
        for i in range(R - 2, -1, -1):
            known_xu_sum = np.dot(U[i, i + 1:], x[i + 1:, 0])
            x[i, 0] = (C[i, 0] - known_xu_sum) / U[i, i]

        print(f"Rozwiązanie dla b[{idx+1}]:\n{x}\n")

    end = time.time()
    total_time = end - start
    print(f" Suma czasów: {total_time:.6f} sekundy")
    return total_time

In [67]:

lu_solve_multiple_b(A, b_list)

Rozwiązanie dla b[1]:
[[-0.1709]
 [-0.2575]
 [ 0.2755]
 [ 0.5809]
 [ 0.4728]
 [-1.1799]
 [ 0.0179]
 [ 0.7365]
 [ 0.1063]
 [ 0.4832]
 [ 0.3182]
 [-0.3988]
 [ 0.1697]]

Rozwiązanie dla b[2]:
[[-0.7267]
 [ 1.4345]
 [-0.0639]
 [-0.2173]
 [-1.1062]
 [ 1.4376]
 [ 0.1101]
 [-0.0906]
 [ 0.7209]
 [-0.7055]
 [-0.5219]
 [-0.1577]
 [ 0.7431]]

Rozwiązanie dla b[3]:
[[ 0.8826]
 [-0.3852]
 [-0.7122]
 [-0.7112]
 [-0.2823]
 [-0.5304]
 [-0.5948]
 [ 0.2248]
 [ 0.5532]
 [ 0.5228]
 [ 0.3034]
 [ 0.5845]
 [ 0.7023]]

Rozwiązanie dla b[4]:
[[-0.1146]
 [ 0.3546]
 [ 0.0785]
 [-0.0962]
 [ 0.4364]
 [ 0.8578]
 [-0.0581]
 [-0.4461]
 [ 0.3108]
 [-0.1039]
 [-0.6393]
 [ 0.1873]
 [ 0.1705]]

 Suma czasów: 0.001007 sekundy


0.0010066032409667969